In [1]:
from gmail_api import init_gmail_service, get_email_messages, get_email_message_details

client_file = 'credentials.json'
service = init_gmail_service(client_file)

gmail v1 service created successfully


In [2]:
query = 'subject:"application" newer_than:1d -subject:"credit card'
emails_last_24hrs = get_email_messages(service, query=query, max_results=None)
emails_last_24hrs

[{'id': '196756254e18feb8', 'threadId': '196756254e18feb8'},
 {'id': '19673f34c3c6c904', 'threadId': '19673f34c3c6c904'}]

In [3]:
def get_emails(messages):
    body = []
    emails = []
    for msg in messages:
        details = get_email_message_details(service,msg['id'])
        if details:
            print(details)
            emails.append(details)

    return emails, body

In [4]:
emails = get_emails(emails_last_24hrs)

{'subject': 'Dhruv, your application was sent to Blockchain Council', 'sender': 'LinkedIn <jobs-noreply@linkedin.com>', 'recipients': 'Dhruv Vaidh <vaidhdhruv@gmail.com>', 'body': 'Your application was sent to Blockchain Council\r\n\r\nAI Intern (Paid)..\r\nBlockchain Council\r\nLos Angeles Metropolitan Area\r\nView job: https://www.linkedin.com/comm/jobs/view/4216015999/?trackingId=ezZvY8jPTuCnF%2FJVXa0bLA%3D%3D&refId=igDKKy%2BeSCKPrrlnI3jDow%3D%3D&lipi=urn%3Ali%3Apage%3Aemail_email_application_confirmation_with_nba_01%3BRTW8FCzdSTO%2Fmm%2FJs8EGdw%3D%3D&midToken=AQG8JIxNcU-3AQ&midSig=2gn2iAefEaZHI1&trk=eml-email_application_confirmation_with_nba_01-job_card-0-view_job&trkEmail=eml-email_application_confirmation_with_nba_01-job_card-0-view_job-null-d7ric0~m9z48v78~on-null-null&eid=d7ric0-m9z48v78-on&otpToken=MTUwYzE3ZTExMjJmYzljM2I2MjQwNGVkNDIxYmUwYjY4OWNlZDk0ODljYTg4NzYxNzljNzAwNjY0ZTVkNWNmMmYzZDNkZmE3MWFkNWRjODk0NTgxZjFmOWE3ODBiNjVmMjBkYjM1Yjg2YTk2OGU4ZWZhMGM2OCwxLDE%3D\r\n\r\n------

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import openai
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the system and human message templates for email analysis
system_message_template = """
You are an assistant that analyzes whether a given email is a job application email.  
- If it is a job application email, extract the following and present them in a JSON object:  
  1. “company_name”  
  2. “job_title”  
  3. “application_status” (Applied, received, rejected, interview invitation, etc.)  
  4. “important_dates” (list of any dates mentioned)  
  5. “action_items” (list of any required follow-up tasks)  

- If it is NOT a job application email, respond with:  
None
"""

human_message_template = """
Analyze the following email:

{email_body}
"""
# Create message templates that will be used in the ChatPromptTemplate in the next cell
system_message = SystemMessage(content=system_message_template)
human_message = HumanMessage(content=human_message_template)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

template = ChatPromptTemplate([
    ("system", system_message_template),
    ("human", human_message_template)])

llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0.2)

prompt_value = template.format_prompt(email_body = emails[0])
messages     = prompt_value.to_messages()
response     = llm(messages=messages)

print(response.content)


ValueError: Invalid template: content='\nYou are an assistant that analyzes whether a given email is a job application email.  \n- If it is a job application email, extract the following and present them in a JSON object:  \n  1. “company_name”  \n  2. “job_title”  \n  3. “application_status” (Applied, received, rejected, interview invitation, etc.)  \n  4. “important_dates” (list of any dates mentioned)  \n  5. “action_items” (list of any required follow-up tasks)  \n\n- If it is NOT a job application email, respond with:  \nNone\n' additional_kwargs={} response_metadata={}

In [ ]:
import os
import re
import json
from typing import Optional, Dict, Any
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


def extract_application_info(raw_response: str) -> Optional[Dict[str, Any]]:
    """
    Given the LLM's raw response, return a Python dict if it contains JSON,
    or None if the response is exactly "None" (case-insensitive).
    """
    text = raw_response.strip()
    # strip markdown code fences
    text = re.sub(r'^```(?:json)?\s*', '', text)
    text = re.sub(r'\s*```$', '', text)
    
    if text.lower() == 'none':
        return None
    
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        # fallback: grab the first {...} block
        m = re.search(r'\{.*\}', text, re.DOTALL)
        if m:
            try:
                return json.loads(m.group())
            except json.JSONDecodeError:
                pass
        raise ValueError(f"Could not parse JSON from response: {raw_response!r}")


def extract_emails(emails):
    # system + human templates
    system_message_template = """
    You are an assistant that analyzes whether a given email is a job application email.  
    - If it is a job application email, extract the following and present them in a JSON object:  
      1. “company_name”  
      2. “job_title”  
      3. “application_status” (Applied, received, rejected, interview invitation, etc.)  
      4. “important_dates” (list of any dates mentioned)  
      5. “action_items” (list of any required follow-up tasks)  

    - If it is NOT a job application email, respond with:
      None
    """

    human_message_template = """
    Analyze the following email:

    {email_body}
    """

    template = ChatPromptTemplate([
        ("system", system_message_template),
        ("human", human_message_template)
    ])

    llm = ChatOpenAI(
        model_name="gpt-4o",
        temperature=0.2
    )
    data = []
    for email in emails:
        prompt_value = template.format_prompt(email_body=email)
        messages     = prompt_value.to_messages()
        response     = llm(messages=messages)
        
        # Clean & parse the LLM's output
        parsed = extract_application_info(response.content)
        data.append(parsed)
        print(parsed)         # dict or None
        print(type(parsed))   # <class 'dict'> or <class 'NoneType'>

/var/folders/_q/6n5wfp6j7gb74y_w3dymgdhc0000gn/T/ipykernel_75224/4176603190.py:66: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


{'company_name': 'Blockchain Council', 'job_title': 'AI Intern', 'application_status': 'Applied', 'important_dates': ['April 27, 2025'], 'action_items': ['Let me know next steps']}
<class 'dict'>


In [13]:
extract_emails(emails)

{'company_name': 'Blockchain Council', 'job_title': 'AI Intern (Paid)', 'application_status': 'Applied', 'important_dates': ['April 27, 2025'], 'action_items': []}
<class 'dict'>
None
<class 'NoneType'>
